In [129]:
rm(list = ls())

In [130]:
# libraries
suppressMessages(library(tidyverse))
library(broom)

In [131]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [132]:
dim(Data)

[1] 1460   81

In [133]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"   "LotArea"      
 [6] "Street"        "Alley"         "LotShape"      "LandContour"   "Utilities"    
[11] "LotConfig"     "LandSlope"     "Neighborhood"  "Condition1"    "Condition2"   
[16] "BldgType"      "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"   "Exterior2nd"  
[26] "MasVnrType"    "MasVnrArea"    "ExterQual"     "ExterCond"     "Foundation"   
[31] "BsmtQual"      "BsmtCond"      "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"   
[36] "BsmtFinType2"  "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"     "X2ndFlrSF"    
[46] "LowQualFinSF"  "GrLivArea"     "BsmtFullBath"  "BsmtHalfBath"  "FullBath"     
[51] "HalfBath"      "BedroomAbvGr"  "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd" 
[56] "Functional"    "Fireplaces"    "FireplaceQu"   "GarageType"

In [134]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [135]:
# numeric data
numData <- Data |> 
  select_if(is.numeric) 

In [136]:
hist(Data$SalePrice)

In [137]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [138]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [139]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [140]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [141]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

Let's impute some of these numerical features

In [142]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [143]:
# list of num featute to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [144]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [145]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [146]:
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  nrow()

[1] 1460

In [147]:
Data |> 
  select_if(is.numeric)  ->numData 
  lm(SalePrice~.,numData) |> 
    summary() |> 
    tidy() |> 
    filter(p.value <=0.5) |> 
    nrow()

[1] 28

28 numerical features have predictive effect on target feature. 

In [148]:
# correlation 
for (feature in names(numData)[-1]){
  if (cor(numData[names(numData)][1],numData[feature]) >= 0.8 ) print(feature)
}

There is no carrelated features in numeric features 

cat var feat engineering  

In [149]:
# n cat features presenting missing values 
sum(cat_skimData$n_missing  !=0)

[1] 16

16 features have missing values or NA

let's use random forest to predict missing values 

to be continued 😉!